# 칼만 필터를 이용한 객체 추적

이 노트북에서는 칼만 필터(Kalman Filter)의 기본 원리를 이해하고, LiDAR 기반 자율주행 시스템에서 
객체 추적에 칼만 필터를 적용하는 방법을 학습합니다.

## 학습 목표

- 칼만 필터의 수학적 기초 이해
- 선형 칼만 필터 구현 및 적용
- LiDAR 객체 추적에서의 칼만 필터 활용
- 확장 칼만 필터(EKF) 및 무향 칼만 필터(UKF) 소개
- 단일 객체 및 다중 객체 추적에 칼만 필터 적용

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import os
import sys
import time
from scipy.linalg import block_diag
from filterpy.kalman import KalmanFilter, ExtendedKalmanFilter, UnscentedKalmanFilter
from filterpy.common import Q_discrete_white_noise
from scipy.optimize import linear_sum_assignment

# 시각화 스타일 설정
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 8)
plt.style.use('ggplot')

## 1. 칼만 필터의 이론적 배경

칼만 필터는 잡음이 있는 측정값으로부터 시스템의 상태를 추정하는 재귀적 알고리즘입니다.
이 알고리즘은 시스템의 이전 상태와 새로운 측정값을 모두 고려하여 현재 상태의 최적 추정치를 계산합니다.

### 칼만 필터의 주요 가정

1. 시스템이 선형적이다 (선형 상태 전이 모델 및 선형 측정 모델)
2. 프로세스 노이즈와 측정 노이즈가 가우시안 분포를 따른다
3. 노이즈는 서로 상관관계가 없다 (독립적)

### 칼만 필터의 주요 단계

1. **예측 단계 (Prediction)**
   - 이전 상태와 제어 입력을 기반으로 현재 상태 예측
   - 상태 공분산 행렬 업데이트

2. **업데이트 단계 (Update)**
   - 칼만 이득 계산
   - 측정값과 예측값의 차이를 반영하여 상태 추정치 업데이트
   - 상태 공분산 행렬 업데이트

### 간단한 1D 칼만 필터 구현

간단한 1차원 위치-속도 추적 문제를 통해 칼만 필터의 기본 원리를 이해해보겠습니다.

In [ ]:
def plot_filter_performance(time_steps, true_states, measurements, estimated_states, covariances):
    """
    칼만 필터의 추정 성능을 시각화
    
    Args:
        time_steps (numpy.ndarray): 시간 스텝
        true_states (numpy.ndarray): 실제 상태 [N, 2] (위치, 속도)
        measurements (numpy.ndarray): 측정 상태 [N, 1] (측정된 위치)
        estimated_states (numpy.ndarray): 추정 상태 [N, 2] (추정된 위치, 속도)
        covariances (numpy.ndarray): 공분산 행렬 [N, 2, 2]
    """
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    
    # 위치 그래프
    axes[0].plot(time_steps, true_states[:, 0], 'g-', label='실제 위치')
    axes[0].plot(time_steps, measurements, 'ro', markersize=4, alpha=0.7, label='측정된 위치')
    axes[0].plot(time_steps, estimated_states[:, 0], 'b-', label='추정된 위치')

    # 2시그마 불확실성 영역 추가
    pos_std = np.sqrt(covariances[:, 0, 0])
    axes[0].fill_between(time_steps, 
                         estimated_states[:, 0] - 2 * pos_std,
                         estimated_states[:, 0] + 2 * pos_std,
                         color='b', alpha=0.2, label='95% 신뢰 구간')
    
    axes[0].set_xlabel('시간 (초)')
    axes[0].set_ylabel('위치 (m)')
    axes[0].set_title('칼만 필터 위치 추정')
    axes[0].legend()
    axes[0].grid(True)
    
    # 속도 그래프
    axes[1].plot(time_steps, true_states[:, 1], 'g-', label='실제 속도')
    axes[1].plot(time_steps, estimated_states[:, 1], 'b-', label='추정된 속도')
    
    # 2시그마 불확실성 영역 추가
    vel_std = np.sqrt(covariances[:, 1, 1])
    axes[1].fill_between(time_steps, 
                         estimated_states[:, 1] - 2 * vel_std,
                         estimated_states[:, 1] + 2 * vel_std,
                         color='b', alpha=0.2, label='95% 신뢰 구간')
    
    axes[1].set_xlabel('시간 (초)')
    axes[1].set_ylabel('속도 (m/s)')
    axes[1].set_title('칼만 필터 속도 추정')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

# 1D 칼만 필터 시뮬레이션
def run_1d_kalman_filter():
    """
    1차원 위치-속도 모델에 대한 칼만 필터 시뮬레이션
    """
    # 시뮬레이션 설정
    dt = 0.1  # 시간 간격 (초)
    T = 10.0  # 총 시뮬레이션 시간 (초)
    time_steps = np.arange(0, T, dt)
    num_steps = len(time_steps)
    
    # 초기 상태 및 동작 모델 설정
    x0 = np.array([0.0, 1.0])  # 초기 위치와 속도
    
    # 프로세스 잡음과 측정 잡음 설정
    process_noise_std = 0.05  # 프로세스 잡음 표준편차
    measurement_noise_std = 0.5  # 측정 잡음 표준편차
    
    # 상태 전이 행렬 (F)
    F = np.array([
        [1.0, dt],
        [0.0, 1.0]
    ])
    
    # 제어 입력 행렬 (B)
    B = np.array([
        [0.5 * dt**2],
        [dt]
    ])
    
    # 측정 행렬 (H) - 위치만 측정
    H = np.array([[1.0, 0.0]])
    
    # 프로세스 잡음 공분산 행렬 (Q)
    q = process_noise_std**2
    Q = np.array([
        [dt**4/4, dt**3/2],
        [dt**3/2, dt**2]
    ]) * q
    
    # 측정 잡음 공분산 행렬 (R)
    R = np.array([[measurement_noise_std**2]])
    
    # 실제 상태, 측정값, 추정 상태를 저장할 배열 초기화
    true_states = np.zeros((num_steps, 2))
    measurements = np.zeros((num_steps, 1))
    estimated_states = np.zeros((num_steps, 2))
    covariances = np.zeros((num_steps, 2, 2))
    
    # 초기 상태 설정
    true_states[0] = x0
    
    # 초기 측정값 생성
    measurements[0] = H @ x0 + np.random.normal(0, measurement_noise_std)
    
    # 칼만 필터 초기화
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.array([measurements[0, 0], 0.0])  # 초기 상태 (위치는 측정값, 속도는 0으로 가정)
    kf.F = F
    kf.H = H
    kf.Q = Q
    kf.R = R
    kf.P = np.eye(2) * 1.0  # 초기 상태 공분산
    
    # 초기 추정값 저장
    estimated_states[0] = kf.x
    covariances[0] = kf.P
    
    # 시뮬레이션
    for i in range(1, num_steps):
        # 실제 상태 업데이트 (제어 입력 없이 등속 운동)
        u = 0.0  # 가속도 입력
        true_states[i] = F @ true_states[i-1] + B.flatten() * u + np.random.normal(0, process_noise_std, 2)
        
        # 측정값 생성
        measurements[i] = H @ true_states[i] + np.random.normal(0, measurement_noise_std)
        
        # 칼만 필터 적용
        kf.predict()
        kf.update(measurements[i])
        
        # 추정값 저장
        estimated_states[i] = kf.x
        covariances[i] = kf.P
    
    # 결과 시각화
    plot_filter_performance(time_steps, true_states, measurements, estimated_states, covariances)
    
    return true_states, measurements, estimated_states, covariances

# 1D 칼만 필터 실행
true_states, measurements, estimated_states, covariances = run_1d_kalman_filter()

## 2. 2D 객체 추적을 위한 칼만 필터

자율주행에서는 주로 2D(or 3D) 공간에서 객체의 위치와 속도를 추적합니다.
여기서는 2D 공간에서 객체 추적을 위한 칼만 필터를 구현해보겠습니다.

In [ ]:
def plot_2d_trajectory(true_states, measurements, estimated_states, covariances, time_steps):
    """
    2D 공간에서의 객체 궤적과 칼만 필터 추정 결과 시각화
    
    Args:
        true_states (numpy.ndarray): 실제 상태 [N, 4] (x, y, vx, vy)
        measurements (numpy.ndarray): 측정 상태 [N, 2] (x, y)
        estimated_states (numpy.ndarray): 추정 상태 [N, 4] (x, y, vx, vy)
        covariances (numpy.ndarray): 공분산 행렬 [N, 4, 4]
    """
    plt.figure(figsize=(10, 8))
    
    # 실제 궤적
    plt.plot(true_states[:, 0], true_states[:, 1], 'g-', linewidth=2, label='실제 궤적')
    
    # 측정값
    plt.scatter(measurements[:, 0], measurements[:, 1], c='r', s=15, alpha=0.7, label='측정값')
    
    # 추정 궤적
    plt.plot(estimated_states[:, 0], estimated_states[:, 1], 'b--', linewidth=2, label='추정 궤적')
    
    # 불확실성 타원 그리기 (몇 개 타임스텝에 대해서만)
    draw_steps = np.linspace(0, len(time_steps)-1, 10).astype(int)
    for i in draw_steps:
        # 공분산 행렬에서 위치 부분만 추출
        pos_cov = covariances[i, :2, :2]
        
        # 고유값과 고유벡터 계산
        eigenvalues, eigenvectors = np.linalg.eig(pos_cov)
        
        # 타원 각도 계산
        angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
        
        # 95% 신뢰 구간을 위한 타원 반경 (2시그마)
        width, height = 2 * np.sqrt(5.991 * eigenvalues)
        
        # 타원 그리기
        ellipse = Ellipse(xy=(estimated_states[i, 0], estimated_states[i, 1]),
                          width=width, height=height, angle=angle,
                          edgecolor='b', fc='none', lw=1, alpha=0.5)
        plt.gca().add_patch(ellipse)
    
    # 시작점과 끝점 표시
    plt.plot(true_states[0, 0], true_states[0, 1], 'go', markersize=10, label='시작점')
    plt.plot(true_states[-1, 0], true_states[-1, 1], 'g*', markersize=10, label='끝점')
    
    plt.xlabel('X 위치 (m)')
    plt.ylabel('Y 위치 (m)')
    plt.title('2D 공간에서의 객체 추적 결과')
    plt.grid(True)
    plt.legend()
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    
    # 시간에 따른 위치 오차 그래프
    plt.figure(figsize=(12, 5))
    
    # 위치 오차 계산
    position_errors = np.sqrt(np.sum((true_states[:, :2] - estimated_states[:, :2])**2, axis=1))
    
    plt.plot(time_steps, position_errors, 'b-', linewidth=2)
    plt.xlabel('시간 (초)')
    plt.ylabel('위치 오차 (m)')
    plt.title('칼만 필터 위치 추정 오차')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# 2D 칼만 필터 구현 및 시뮬레이션
def run_2d_kalman_filter(model_type='CV'):
    """
    2D 칼만 필터 시뮬레이션
    
    Args:
        model_type (str): 운동 모델 유형 ('CV': 등속 모델, 'CA': 등가속도 모델)
        
    Returns:
        tuple: (true_states, measurements, estimated_states, covariances, time_steps)
    """
    # 시뮬레이션 설정
    dt = 0.1  # 시간 간격 (초)
    T = 10.0  # 총 시뮬레이션 시간 (초)
    time_steps = np.arange(0, T, dt)
    num_steps = len(time_steps)
    
    # 프로세스 잡음과 측정 잡음 설정
    process_noise_std = 0.1  # 프로세스 잡음 표준편차
    measurement_noise_std = 0.5  # 측정 잡음 표준편차
    
    if model_type == 'CV':  # 등속 모델 (Constant Velocity)
        # 초기 상태: [x, y, vx, vy]
        x0 = np.array([0.0, 0.0, 2.0, 1.0])
        dim_x = 4
        
        # 상태 전이 행렬 (F)
        F = np.array([
            [1, 0, dt, 0],
            [0, 1, 0, dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        
        # 측정 행렬 (H) - 위치만 측정
        H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])
        
        # 프로세스 잡음 공분산 행렬 (Q)
        q = process_noise_std**2
        Q = np.array([
            [dt**4/4, 0, dt**3/2, 0],
            [0, dt**4/4, 0, dt**3/2],
            [dt**3/2, 0, dt**2, 0],
            [0, dt**3/2, 0, dt**2]
        ]) * q
        
    elif model_type == 'CA':  # 등가속도 모델 (Constant Acceleration)
        # 초기 상태: [x, y, vx, vy, ax, ay]
        x0 = np.array([0.0, 0.0, 2.0, 1.0, 0.1, 0.1])
        dim_x = 6
        
        # 상태 전이 행렬 (F)
        F = np.array([
            [1, 0, dt, 0, dt**2/2, 0],
            [0, 1, 0, dt, 0, dt**2/2],
            [0, 0, 1, 0, dt, 0],
            [0, 0, 0, 1, 0, dt],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1]
        ])
        
        # 측정 행렬 (H) - 위치만 측정
        H = np.array([
            [1, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0]
        ])
        
        # 프로세스 잡음 공분산 행렬 (Q) - 가속도 변화에 대한 불확실성
        q = process_noise_std**2
        Q = np.zeros((6, 6))
        Q[4:, 4:] = np.array([[dt**2, 0], [0, dt**2]]) * q
        
    else:
        raise ValueError(f"지원하지 않는 모델 유형: {model_type}")
    
    # 측정 잡음 공분산 행렬 (R)
    R = np.eye(2) * measurement_noise_std**2
    
    # 실제 상태, 측정값, 추정 상태를 저장할 배열 초기화
    true_states = np.zeros((num_steps, dim_x))
    measurements = np.zeros((num_steps, 2))
    estimated_states = np.zeros((num_steps, dim_x))
    covariances = np.zeros((num_steps, dim_x, dim_x))
    
    # 초기 상태 설정
    true_states[0] = x0
    
    # 초기 측정값 생성
    measurements[0] = H @ x0 + np.random.normal(0, measurement_noise_std, 2)
    
    # 칼만 필터 초기화
    kf = KalmanFilter(dim_x=dim_x, dim_z=2)
    kf.x = np.zeros(dim_x)  # 초기 상태
    kf.x[:2] = measurements[0]  # 위치는 측정값으로 초기화
    if model_type == 'CV':
        kf.x[2:] = [1.0, 1.0]  # 초기 속도는 적당히 가정
    else:  # CA 모델
        kf.x[2:4] = [1.0, 1.0]  # 초기 속도
        kf.x[4:] = [0.0, 0.0]  # 초기 가속도
    
    kf.F = F
    kf.H = H
    kf.Q = Q
    kf.R = R
    kf.P = np.eye(dim_x) * 1.0  # 초기 상태 공분산
    
    # 초기 추정값 저장
    estimated_states[0] = kf.x
    covariances[0] = kf.P
    
    # 시뮬레이션 - 곡선 궤적 생성
    angle = 0
    for i in range(1, num_steps):
        # 실제 상태 업데이트 - 약간의 원형 궤적
        # 회전 각도 증가
        angle += 0.05
        
        if model_type == 'CV':
            # 속도 벡터 회전
            vx = 2.0 * np.cos(angle)
            vy = 2.0 * np.sin(angle)
            
            # 이전 위치에서 속도만큼 이동
            prev_x, prev_y = true_states[i-1, 0], true_states[i-1, 1]
            new_x = prev_x + vx * dt
            new_y = prev_y + vy * dt
            
            true_states[i] = [new_x, new_y, vx, vy]
            true_states[i] += np.random.normal(0, process_noise_std, 4)
            
        else:  # CA 모델
            # 속도와 가속도 벡터 회전
            vx = 2.0 * np.cos(angle)
            vy = 2.0 * np.sin(angle)
            ax = -2.0 * np.sin(angle) * 0.05  # 각속도 * 속도
            ay = 2.0 * np.cos(angle) * 0.05
            
            # 이전 위치에서 속도와 가속도만큼 이동
            prev_x, prev_y = true_states[i-1, 0], true_states[i-1, 1]
            prev_vx, prev_vy = true_states[i-1, 2], true_states[i-1, 3]
            
            new_x = prev_x + prev_vx * dt + 0.5 * ax * dt**2
            new_y = prev_y + prev_vy * dt + 0.5 * ay * dt**2
            new_vx = prev_vx + ax * dt
            new_vy = prev_vy + ay * dt
            
            true_states[i] = [new_x, new_y, new_vx, new_vy, ax, ay]
            true_states[i] += np.random.normal(0, process_noise_std, 6)
        
        # 측정값 생성
        measurements[i] = H @ true_states[i] + np.random.normal(0, measurement_noise_std, 2)
        
        # 칼만 필터 적용
        kf.predict()
        kf.update(measurements[i])
        
        # 추정값 저장
        estimated_states[i] = kf.x
        covariances[i] = kf.P
    
    return true_states, measurements, estimated_states, covariances, time_steps

# 2D 등속 모델 (CV) 칼만 필터 실행
true_states_cv, measurements_cv, estimated_states_cv, covariances_cv, time_steps_cv = run_2d_kalman_filter('CV')

# 결과 시각화
plot_2d_trajectory(true_states_cv, measurements_cv, estimated_states_cv, covariances_cv, time_steps_cv)

# 2D 등가속도 모델 (CA) 칼만 필터 실행
true_states_ca, measurements_ca, estimated_states_ca, covariances_ca, time_steps_ca = run_2d_kalman_filter('CA')

# 결과 시각화
plot_2d_trajectory(true_states_ca[:, :4], measurements_ca, estimated_states_ca[:, :4], covariances_ca[:, :4, :4], time_steps_ca)


## 3. LiDAR 객체 추적에서의 칼만 필터 활용

LiDAR 데이터를 사용한 객체 추적에서는 검출된 객체의 바운딩 박스 중심을 측정값으로 활용하여
칼만 필터를 적용합니다. 여기서는 간단한 시뮬레이션을 통해 자율주행 환경에서의 객체 추적을 구현해보겠습니다.


In [ ]:
def create_turning_trajectory(duration=10.0, dt=0.1, turn_rate=0.05, noise_std=0.02, initial_state=None):
    """
    회전하는 객체의 궤적 생성
    
    Args:
        duration (float): 시뮬레이션 시간 (초)
        dt (float): 시간 간격 (초)
        turn_rate (float): 회전 속도 (rad/s)
        noise_std (float): 위치 노이즈 표준편차
        initial_state (numpy.ndarray): 초기 상태 [x, y, vx, vy]
        
    Returns:
        tuple: (true_states, measurements, time_steps)
    """
    # 시간 스텝 설정
    time_steps = np.arange(0, duration, dt)
    num_steps = len(time_steps)
    
    # 초기 상태 설정
    if initial_state is None:
        initial_state = np.array([0.0, 0.0, 2.0, 0.0])  # 기본값: 원점에서 출발, x방향 속도 2 m/s
    
    # 측정 노이즈 설정
    measurement_noise_std = 0.2  # 측정 노이즈 표준편차
    
    # 상태와 측정값 저장할 배열 초기화
    true_states = np.zeros((num_steps, 4))  # [x, y, vx, vy]
    measurements = np.zeros((num_steps, 2))  # [x, y]
    
    # 초기 상태 설정
    true_states[0] = initial_state
    
    # 초기 측정값 생성
    measurements[0] = true_states[0, :2] + np.random.normal(0, measurement_noise_std, 2)
    
    # 회전 각도 초기화
    angle = np.arctan2(initial_state[3], initial_state[2])
    speed = np.sqrt(initial_state[2]**2 + initial_state[3]**2)
    
    # 시뮬레이션
    for i in range(1, num_steps):
        # 회전 각도 업데이트
        angle += turn_rate * dt
        
        # 속도 벡터 업데이트
        vx = speed * np.cos(angle)
        vy = speed * np.sin(angle)
        
        # 위치 업데이트
        prev_x, prev_y = true_states[i-1, 0], true_states[i-1, 1]
        new_x = prev_x + vx * dt
        new_y = prev_y + vy * dt
        
        # 프로세스 노이즈 추가
        process_noise = np.random.normal(0, noise_std, 4)
        true_states[i] = np.array([new_x, new_y, vx, vy]) + process_noise
        
        # 측정값 생성 (위치에 노이즈 추가)
        measurements[i] = true_states[i, :2] + np.random.normal(0, measurement_noise_std, 2)
    
    return true_states, measurements, time_steps

def simulate_lidar_detection(true_states, detection_probability=0.9, false_detection_rate=0.1, 
                           range_limit=50.0, position_noise_std=0.3):
    """
    LiDAR 객체 검출 시뮬레이션
    
    Args:
        true_states (numpy.ndarray): 실제 객체 상태 [N, 4] (x, y, vx, vy)
        detection_probability (float): 객체 검출 확률
        false_detection_rate (float): 허위 검출 비율
        range_limit (float): LiDAR 범위 제한 (m)
        position_noise_std (float): 위치 측정 노이즈 표준편차
        
    Returns:
        list: 프레임별 객체 검출 리스트 [프레임][검출_인덱스][x, y]
    """
    num_frames = len(true_states)
    detections = []
    
    for i in range(num_frames):
        frame_detections = []
        
        # 실제 객체 위치
        pos_x, pos_y = true_states[i, 0], true_states[i, 1]
        
        # 객체가 범위 내에 있고 검출 확률을 만족하는 경우
        distance = np.sqrt(pos_x**2 + pos_y**2)
        if distance < range_limit and np.random.rand() < detection_probability:
            # 위치에 노이즈 추가
            detected_x = pos_x + np.random.normal(0, position_noise_std)
            detected_y = pos_y + np.random.normal(0, position_noise_std)
            frame_detections.append([detected_x, detected_y])
        
        # 허위 검출 추가
        num_false_detections = int(np.random.poisson(false_detection_rate))
        for _ in range(num_false_detections):
            # 임의의 위치에 허위 검출 생성
            false_x = np.random.uniform(-range_limit, range_limit)
            false_y = np.random.uniform(-range_limit, range_limit)
            frame_detections.append([false_x, false_y])
        
        detections.append(frame_detections)
    
    return detections

class KalmanObjectTracker:
    """
    칼만 필터를 사용한 객체 추적기
    """
    def __init__(self, dt, process_noise_std, measurement_noise_std, initial_state=None):
        # 칼만 필터 초기화
        self.kf = KalmanFilter(dim_x=4, dim_z=2)
        
        # 초기 상태 설정
        if initial_state is not None:
            self.kf.x = initial_state
        else:
            self.kf.x = np.zeros(4)
        
        # 상태 전이 행렬 (F) - 등속 모델
        self.kf.F = np.array([
            [1, 0, dt, 0],
            [0, 1, 0, dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        
        # 측정 행렬 (H) - 위치만 측정
        self.kf.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])
        
        # 프로세스 잡음 공분산 행렬 (Q)
        q = process_noise_std**2
        self.kf.Q = np.array([
            [dt**4/4, 0, dt**3/2, 0],
            [0, dt**4/4, 0, dt**3/2],
            [dt**3/2, 0, dt**2, 0],
            [0, dt**3/2, 0, dt**2]
        ]) * q
        
        # 측정 잡음 공분산 행렬 (R)
        self.kf.R = np.eye(2) * measurement_noise_std**2
        
        # 상태 공분산 행렬 (P) 초기화
        self.kf.P = np.eye(4) * 1.0
        
        # 추적 이력
        self.history = [self.kf.x.copy()]
        self.covariance_history = [self.kf.P.copy()]
        
        # 추적 상태
        self.age = 0
        self.missed_detections = 0
        self.max_missed_detections = 5  # 최대 몇 프레임까지 검출 없이 추적을 유지할지
    
    def predict(self):
        """
        상태 예측
        """
        self.kf.predict()
        self.age += 1
        
        return self.kf.x
    
    def update(self, measurement):
        """
        측정값으로 상태 업데이트
        
        Args:
            measurement (numpy.ndarray): 측정값 [x, y]
        """
        self.kf.update(measurement)
        self.missed_detections = 0
        
        # 이력 추가
        self.history.append(self.kf.x.copy())
        self.covariance_history.append(self.kf.P.copy())
        
        return self.kf.x
    
    def missed_update(self):
        """
        측정값 없이 상태 유지 (예측만 수행)
        """
        self.missed_detections += 1
        
        # 이력 추가
        self.history.append(self.kf.x.copy())
        self.covariance_history.append(self.kf.P.copy())
        
        return self.kf.x, self.missed_detections > self.max_missed_detections
    
    def get_state(self):
        """
        현재 상태 반환
        """
        return self.kf.x
    
    def get_position(self):
        """
        현재 위치 반환
        """
        return self.kf.x[:2]
    
    def get_velocity(self):
        """
        현재 속도 반환
        """
        return self.kf.x[2:4]
    
    def get_covariance(self):
        """
        현재 공분산 행렬 반환
        """
        return self.kf.P
    
    def get_position_covariance(self):
        """
        위치 공분산 반환
        """
        return self.kf.P[:2, :2]

def associate_detections_to_trackers(detections, trackers, threshold=1.0):
    """
    검출과 추적기 간의 연관성 찾기
    
    Args:
        detections (list): 검출 리스트 [[x1, y1], [x2, y2], ...]
        trackers (list): 추적기 리스트 [tracker1, tracker2, ...]
        threshold (float): 최대 거리 임계값
        
    Returns:
        tuple: (matches, unmatched_detections, unmatched_trackers)
    """
    if len(trackers) == 0:
        return [], list(range(len(detections))), []
    
    if len(detections) == 0:
        return [], [], list(range(len(trackers)))
    
    # 거리 행렬 계산
    distance_matrix = np.zeros((len(detections), len(trackers)))
    
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            trk_pos = trk.get_position()
            distance = np.linalg.norm(det - trk_pos)
            distance_matrix[d, t] = distance
    
    # 헝가리안 알고리즘으로 최적 할당 찾기
    from scipy.optimize import linear_sum_assignment
    rows, cols = linear_sum_assignment(distance_matrix)
    
    # 임계값보다 거리가 큰 경우 매칭에서 제외
    matches = []
    unmatched_detections = []
    unmatched_trackers = list(range(len(trackers)))
    
    for r, c in zip(rows, cols):
        if distance_matrix[r, c] <= threshold:
            matches.append((r, c))
            if c in unmatched_trackers:
                unmatched_trackers.remove(c)
        else:
            unmatched_detections.append(r)
    
    # 매칭되지 않은 검출 추가
    for d in range(len(detections)):
        if d not in rows:
            unmatched_detections.append(d)
    
    return matches, unmatched_detections, unmatched_trackers

def plot_tracking_results(true_states, detections, tracked_states, time_steps):
    """
    객체 추적 결과 시각화
    
    Args:
        true_states (numpy.ndarray): 실제 상태 [N, 4] (x, y, vx, vy)
        detections (list): 프레임별 검출 리스트 [프레임][검출_인덱스][x, y]
        tracked_states (list): 프레임별 추적 상태 리스트 [프레임][추적기_인덱스][상태]
        time_steps (numpy.ndarray): 시간 스텝
    """
    plt.figure(figsize=(12, 10))
    
    # 실제 궤적
    plt.plot(true_states[:, 0], true_states[:, 1], 'g-', linewidth=2, label='실제 궤적')
    
    # 검출 플롯
    for i, frame_dets in enumerate(detections):
        for det in frame_dets:
            plt.scatter(det[0], det[1], c='r', s=20, alpha=0.4)
    
    # 추적 결과 플롯
    tracked_x = []
    tracked_y = []
    
    for i, frame_tracks in enumerate(tracked_states):
        for track in frame_tracks:
            tracked_x.append(track[0])
            tracked_y.append(track[1])
            
            # 추정 상태의 불확실성 타원 그리기 (몇 개 프레임에 대해서만)
            if i % 10 == 0:
                pos_cov = track[-1][:2, :2]  # 위치 공분산
                
                # 고유값과 고유벡터 계산
                eigenvalues, eigenvectors = np.linalg.eig(pos_cov)
                
                # 타원 각도 계산
                angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
                
                # 95% 신뢰 구간을 위한 타원 반경 (2시그마)
                width, height = 2 * np.sqrt(5.991 * eigenvalues)
                
                # 타원 그리기
                ellipse = Ellipse(xy=(track[0], track[1]),
                                width=width, height=height, angle=angle,
                                edgecolor='b', fc='none', lw=1, alpha=0.5)
                plt.gca().add_patch(ellipse)
    
    plt.plot(tracked_x, tracked_y, 'b--', linewidth=2, label='추적 궤적')
    
    # 시작점과 끝점 표시
    plt.plot(true_states[0, 0], true_states[0, 1], 'go', markersize=10, label='시작점')
    plt.plot(true_states[-1, 0], true_states[-1, 1], 'g*', markersize=10, label='끝점')
    
    plt.xlabel('X 위치 (m)')
    plt.ylabel('Y 위치 (m)')
    plt.title('LiDAR 객체 추적 결과')
    plt.grid(True)
    plt.legend()
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    
    # 시간에 따른 위치 오차 그래프
    plt.figure(figsize=(12, 5))
    
    # 위치 오차 계산
    position_errors = []
    for i, frame_tracks in enumerate(tracked_states):
        if frame_tracks:  # 추적기가 있는 경우
            # 첫 번째 추적기의 위치 사용
            track_pos = np.array([frame_tracks[0][0], frame_tracks[0][1]])
            true_pos = true_states[i, :2]
            error = np.linalg.norm(track_pos - true_pos)
            position_errors.append(error)
        else:
            position_errors.append(np.nan)  # 추적기가 없는 경우
    
    plt.plot(time_steps[:len(position_errors)], position_errors, 'b-', linewidth=2)
    plt.xlabel('시간 (초)')
    plt.ylabel('위치 오차 (m)')
    plt.title('칼만 필터 위치 추정 오차')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def run_multi_object_tracker(detections, dt, process_noise_std=0.1, measurement_noise_std=0.3):
    """
    다중 객체 추적 시뮬레이션
    
    Args:
        detections (list): 프레임별 검출 리스트 [프레임][검출_인덱스][x, y]
        dt (float): 시간 간격
        process_noise_std (float): 프로세스 노이즈 표준편차
        measurement_noise_std (float): 측정 노이즈 표준편차
        
    Returns:
        list: 프레임별 추적 상태 리스트 [프레임][추적기_인덱스][상태]
    """
    trackers = []  # 활성 추적기 리스트
    tracked_states = []  # 각 프레임의 추적 상태를 저장
    
    for frame_idx, frame_detections in enumerate(detections):
        # 현재 프레임의 추적 상태를 저장할 리스트
        current_tracked_states = []
        
        # 기존 추적기의 상태 예측
        for tracker in trackers:
            tracker.predict()
        
        # 검출과 추적기 연관성 찾기
        det_array = np.array(frame_detections) if len(frame_detections) > 0 else np.empty((0, 2))
        matches, unmatched_detections, unmatched_trackers = associate_detections_to_trackers(
            det_array, trackers)
        
        # 매칭된 검출로 추적기 업데이트
        for det_idx, trk_idx in matches:
            trackers[trk_idx].update(det_array[det_idx])
        
        # 매칭되지 않은 추적기 업데이트 (측정값 없이)
        trackers_to_remove = []
        for idx in unmatched_trackers:
            _, should_remove = trackers[idx].missed_update()
            if should_remove:
                trackers_to_remove.append(idx)
        
        # 제거할 추적기 제거 (역순으로)
        for idx in sorted(trackers_to_remove, reverse=True):
            trackers.pop(idx)
        
        # 새로운 검출로 추적기 생성
        for idx in unmatched_detections:
            # 초기 상태: [x, y, 0, 0] - 위치는 검출된 위치, 속도는 0으로 초기화
            initial_state = np.array([det_array[idx][0], det_array[idx][1], 0, 0])
            new_tracker = KalmanObjectTracker(dt, process_noise_std, measurement_noise_std, initial_state)
            trackers.append(new_tracker)
        
        # 현재 프레임의 모든 추적기 상태 저장
        for tracker in trackers:
            state = tracker.get_state()
            covariance = tracker.get_covariance()
            # 상태와 공분산 함께 저장
            current_tracked_states.append(np.concatenate([state, covariance.flatten()]))
        
        tracked_states.append(current_tracked_states)
    
    return tracked_states

# 객체 궤적 생성
duration = 10.0
dt = 0.1
true_states, measurements, time_steps = create_turning_trajectory(
    duration=duration, dt=dt, turn_rate=0.1, noise_std=0.05)

# LiDAR 검출 시뮬레이션
detections = simulate_lidar_detection(
    true_states, detection_probability=0.8, false_detection_rate=0.2)

# 객체 추적 실행
tracked_states = run_multi_object_tracker(
    detections, dt, process_noise_std=0.1, measurement_noise_std=0.3)

# 결과 시각화
plot_tracking_results(true_states, detections, tracked_states, time_steps)

## 4. 확장 칼만 필터(EKF)와 무향 칼만 필터(UKF)

실제 자율주행 시스템에서는 비선형 모델을 다루는 경우가 많습니다.
비선형 시스템에서는 확장 칼만 필터(EKF)나 무향 칼만 필터(UKF) 등을 활용합니다.